### Upper-Confidence-Bound (UCB) Action Selection

$A_t = \underset{a}{\operatorname{argmax}} [Q_t(a)+c\sqrt{\frac{\ln t}{N_t(a)}}]$ <br/><br/>
The square root term represents the uncertainty or variance in the estimate of $a$'s value. The term behind $\operatorname{argmax}$ is thus a sort of upper bound on the possible true value of $a$ with $c$ determining the confidence level. <br/>
Each time $a$ is selected the uncertainty is reduced: $N_t(a)$ increments. On the other hand, if an action other than $a$ is selected, $t$ increases but $N_t(a)$ does not; the uncertainty estimate increases. The use of $\ln$ means the increases get smaller over time.

### Gradient Bandit Algorithm

Update a preference $H_t(a)$ of an action $a$ instead of reward. $H_t(a)$ has no interpretation in terms of reward. Action probabilities are modelled by a soft-max distribution (i.e., Gib or Boltmann distribution): <br/>

$Pr\{A_t = a\} = \frac{e^{H_t(a)}}{\sum_{b=1}^{k} e^{H_t(b)} } = \pi_t(a)$ represents the probability of taking action $a$ at time $t$.  <br/><br/>

On each step, after selecting action $A_t$ and receiving the reward $R_t$, preferences are updated by: <br/>
$H_{t+1}(A_t) = H_t(A_t)+\alpha(R_t - Q_t)(1-\pi_t(A_t))$ <br/>
where $Q_t$ is the sample mean computed as in $\epsilon$-greedy. For all $a \ne A_t$, <br/>
$H_{t+1}(a)=H_t(a)-\alpha(R_t - Q_t)\pi_t(a)$

In [1]:
class NonstationaryAction:
    '''
        Actions of which the expected rewards (true values) change over time.
        In this case, it makes more sense to give more weight to recent rewards
        than to long-past reqards, acheived by using a constant step-size within (0,1]
        in the update rule.
    '''
    
    def __init__(self, true_mean, initial_value = 0, step_size = 0.1, random_walk_std = 0.01):
        '''
            random_walk_std is a standard devitation of a Gaussian distribution
            for modifying the true expected reward associated with each action
        '''
        # action value = expected reward
        self.expected_mean = true_mean
        # estimate of the action value = sample-mean of rewards
        self.sample_mean = initial_value
        # preference
        self.preference = 0
        self.is_selected = False
        # number of times the action has been selected
        self.number_of_times = 0 
        # a constant step-size for the update rule
        self.step_size = step_size
        self.random_walk_std = random_walk_std
        
    
    def copy(self, action):
        
        self.expected_mean = action.expected_mean
        self.sample_mean = action.sample_mean
        self.number_of_times = action.number_of_times
        self.step_size = action.step_size
        self.random_walk_std = action.random_walk_std
        
    def set_initial_value(self, initial_value):
        self.sample_mean = initial_value
        
    def act(self):
        return np.random.normal(self.expected_mean,self.random_walk_std,1)
    
    def nonstationary(self):
        self.expected_mean += np.random.normal(0,1.,1)
    
    def update_value(self, reward):
        '''
            This function computes a sample mean of rewards after an action has been performed
            i.e., recompute the sample mean given a new reward 
        '''
        self.number_of_times += 1
        # the updte rule
        

In [ ]:
def nonstationary_gradient_bandit(action_space, max_num_step = 100, epsilon = 0.1):
    '''
        This algorithm is for actions whose assocaited true expected rewards are nonstationary or vary from time step to time step
        The algorithm selects an action having the highest value (sample-averaged reward) most of the times,
        and with a probability of \epsilon (probability of exploration) selects an action randomly with equal probability 
        (independence of its sample-averaged reward). 
        The typical value for \epsilon is 0.05 or 0.1
    '''
    
    def random_walk_action_value(action_space):
        #print('Expected mean ', [a.expected_mean for a in action_space])
        for a in action_space:
            a.nonstationary()
        #print('After changes, Expected mean ', [a.expected_mean for a in action_space])
        
    acquired_rewards = [] # acquired rewards at each time point

    for n in range(max_num_step):
        
        prob = np.random.uniform(0.,1.,1)
        
        # simulate nonstationarity
        random_walk_action_value(action_space)
        
        # select action according to probability of exploration
        action_id = None
        # explore other actions with probability of \epsilon
        if prob <= epsilon: 
            action_id = np.random.choice(len(action_space))
        else: # pick the best action with probability 1-\epsilon
            est_mean_rewards = np.asarray([a.sample_mean for a in action_space])
            action_id = np.argmax(est_mean_rewards)
            
        selected_action = action_space[action_id]
        reward = selected_action.act()
        selected_action.update_value(reward)
        
        # collect rewards for plotting
        acquired_rewards.append(reward) 
        
    # cumulative rewards at each time point
    cumulative_rewards = np.cumsum(acquired_rewards)/(np.arange(max_num_step, dtype = np.float32)+1.0)
    
    return acquired_rewards, cumulative_rewards


In [ ]:
# create a list of actions
nonstationary_action_space = []
for i in range(20):
    nonstationary_action_space.append(NonstationaryAction(np.random.uniform(0., 2., 1)))

_, cumulative_rewards01 = nonstationary_bandit(nonstationary_action_space, 10000, 0.1)
_, cumulative_rewards005 = nonstationary_bandit(nonstationary_action_space, 10000, 0.05)
_, cumulative_rewards001 = nonstationary_bandit(nonstationary_action_space, 10000, 0.01)
_, cumulative_rewards0 = nonstationary_bandit(nonstationary_action_space, 10000, 0.) # greedy